In [4]:
import numpy as np
from itertools import product
import math
import random
import copy

#### Only H

In [28]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # The mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J for the termination condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))

            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()
            
            epoch += 1

        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
        
    def random_walk(self, max_steps):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        burn_in_len = 100
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            # If delta_energy<0, accept the spin flip, energetically favourable
            if delta_energy>0:  # Ohterwise, check if transition probability p < uniform r

                p = math.exp(-delta_energy)
                r = random.random()

                if r>p: # If transition probability is lower than r, refuse the update, otherwise accept
                    sigma_t = copy.deepcopy(sigma_0)
                    energy_t = energy_0
            
            # Save the state if the burn-in phase has passed
            if i > burn_in_len:
                model_activations_list.append(sigma_t)

            sigma_0 = copy.deepcopy(sigma_t)
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, random_walk_len = 100):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(random_walk_len)   
            model_activations_matrix = np.array(model_activations_list)

            # Calculate the data expectations
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of H for the stop condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))
            
            print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
            print(model_mean, 'Average Model')
            print(self.data_mean, 'Average Data')
            print(self.data_mean - model_mean, 'Diff')
            print()
            
            epoch +=1
        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')


In [29]:
activations_list = [[1], [1], [-1], [-1], [1], [-1], [1], [1]]
s_ising = SimpleIsingModel(activations_list, lr=0.0005)


In [30]:
s_ising.train_metropolis( max_epochs = 2500, random_walk_len = 400)

Epoch 1 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 2 TotalParamVariation 0.00012333
[0.00334448] Average Model
[0.25] Average Data
[0.24665552] Diff

Epoch 3 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 4 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 5 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 6 TotalParamVariation 0.00012333
[0.00334448] Average Model
[0.25] Average Data
[0.24665552] Diff

Epoch 7 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 8 TotalParamVariation 0.00012667
[-0.00334448] Average Model
[0.25] Average Data
[0.25334448] Diff

Epoch 9 TotalParamVariation 0.00012333
[0.00334448] Average Model
[0.25] Average Data
[0.24665552] Diff

Epoch 10 TotalParamVariation 0.00012333
[0.003344

Epoch 108 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 109 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 110 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 111 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 112 TotalParamVariation 0.00011664
[0.01672241] Average Model
[0.25] Average Data
[0.23327759] Diff

Epoch 113 TotalParamVariation 0.00012333
[0.00334448] Average Model
[0.25] Average Data
[0.24665552] Diff

Epoch 114 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 115 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 116 TotalParamVariation 0.00011664
[0.01672241] Average Model
[0.25] Average Data
[0.23327759] Diff

Epoch 117 TotalParamVariation 0.00011

Epoch 227 TotalParamVariation 0.00010995
[0.03010033] Average Model
[0.25] Average Data
[0.21989967] Diff

Epoch 228 TotalParamVariation 0.00011664
[0.01672241] Average Model
[0.25] Average Data
[0.23327759] Diff

Epoch 229 TotalParamVariation 0.00010995
[0.03010033] Average Model
[0.25] Average Data
[0.21989967] Diff

Epoch 230 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 231 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 232 TotalParamVariation 9.992e-05
[0.05016722] Average Model
[0.25] Average Data
[0.19983278] Diff

Epoch 233 TotalParamVariation 0.00011998
[0.01003344] Average Model
[0.25] Average Data
[0.23996656] Diff

Epoch 234 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 235 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 236 TotalParamVariation 0.000109

Epoch 329 TotalParamVariation 0.00010326
[0.04347826] Average Model
[0.25] Average Data
[0.20652174] Diff

Epoch 330 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 331 TotalParamVariation 9.992e-05
[0.05016722] Average Model
[0.25] Average Data
[0.19983278] Diff

Epoch 332 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 333 TotalParamVariation 0.00010995
[0.03010033] Average Model
[0.25] Average Data
[0.21989967] Diff

Epoch 334 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 335 TotalParamVariation 0.00010995
[0.03010033] Average Model
[0.25] Average Data
[0.21989967] Diff

Epoch 336 TotalParamVariation 0.00011664
[0.01672241] Average Model
[0.25] Average Data
[0.23327759] Diff

Epoch 337 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Data
[0.22658863] Diff

Epoch 338 TotalParamVariation 0.000103

[0.0367893] Average Model
[0.25] Average Data
[0.2132107] Diff

Epoch 425 TotalParamVariation 9.657e-05
[0.05685619] Average Model
[0.25] Average Data
[0.19314381] Diff

Epoch 426 TotalParamVariation 0.00010995
[0.03010033] Average Model
[0.25] Average Data
[0.21989967] Diff

Epoch 427 TotalParamVariation 9.992e-05
[0.05016722] Average Model
[0.25] Average Data
[0.19983278] Diff

Epoch 428 TotalParamVariation 9.657e-05
[0.05685619] Average Model
[0.25] Average Data
[0.19314381] Diff

Epoch 429 TotalParamVariation 9.992e-05
[0.05016722] Average Model
[0.25] Average Data
[0.19983278] Diff

Epoch 430 TotalParamVariation 0.00010661
[0.0367893] Average Model
[0.25] Average Data
[0.2132107] Diff

Epoch 431 TotalParamVariation 0.00010326
[0.04347826] Average Model
[0.25] Average Data
[0.20652174] Diff

Epoch 432 TotalParamVariation 0.00010326
[0.04347826] Average Model
[0.25] Average Data
[0.20652174] Diff

Epoch 433 TotalParamVariation 0.00011329
[0.02341137] Average Model
[0.25] Average Dat

...

[0.14381271] Average Model
[0.25] Average Data
[0.10618729] Diff

Epoch 1973 TotalParamVariation 3.637e-05
[0.17725753] Average Model
[0.25] Average Data
[0.07274247] Diff

Epoch 1974 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 1975 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 1976 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 1977 TotalParamVariation 3.303e-05
[0.18394649] Average Model
[0.25] Average Data
[0.06605351] Diff

Epoch 1978 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 1979 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 1980 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 1981 TotalParamVariation 5.309e-05
[0.14381271] Average Model
[0.25] Avera

Epoch 2068 TotalParamVariation 6.313e-05
[0.12374582] Average Model
[0.25] Average Data
[0.12625418] Diff

Epoch 2069 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 2070 TotalParamVariation 6.647e-05
[0.11705686] Average Model
[0.25] Average Data
[0.13294314] Diff

Epoch 2071 TotalParamVariation 5.978e-05
[0.13043478] Average Model
[0.25] Average Data
[0.11956522] Diff

Epoch 2072 TotalParamVariation 2.968e-05
[0.19063545] Average Model
[0.25] Average Data
[0.05936455] Diff

Epoch 2073 TotalParamVariation 5.309e-05
[0.14381271] Average Model
[0.25] Average Data
[0.10618729] Diff

Epoch 2074 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2075 TotalParamVariation 4.64e-05
[0.15719064] Average Model
[0.25] Average Data
[0.09280936] Diff

Epoch 2076 TotalParamVariation 5.644e-05
[0.13712375] Average Model
[0.25] Average Data
[0.11287625] Diff

Epoch 2077 TotalParamVariation 4.975e-05

Epoch 2172 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 2173 TotalParamVariation 3.303e-05
[0.18394649] Average Model
[0.25] Average Data
[0.06605351] Diff

Epoch 2174 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 2175 TotalParamVariation 3.303e-05
[0.18394649] Average Model
[0.25] Average Data
[0.06605351] Diff

Epoch 2176 TotalParamVariation 6.313e-05
[0.12374582] Average Model
[0.25] Average Data
[0.12625418] Diff

Epoch 2177 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 2178 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 2179 TotalParamVariation 5.644e-05
[0.13712375] Average Model
[0.25] Average Data
[0.11287625] Diff

Epoch 2180 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2181 TotalParamVariation 4.975e-0

[0.25] Average Data
[0.0861204] Diff

Epoch 2280 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2281 TotalParamVariation 3.303e-05
[0.18394649] Average Model
[0.25] Average Data
[0.06605351] Diff

Epoch 2282 TotalParamVariation 4.64e-05
[0.15719064] Average Model
[0.25] Average Data
[0.09280936] Diff

Epoch 2283 TotalParamVariation 4.975e-05
[0.15050167] Average Model
[0.25] Average Data
[0.09949833] Diff

Epoch 2284 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2285 TotalParamVariation 3.303e-05
[0.18394649] Average Model
[0.25] Average Data
[0.06605351] Diff

Epoch 2286 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2287 TotalParamVariation 5.309e-05
[0.14381271] Average Model
[0.25] Average Data
[0.10618729] Diff

Epoch 2288 TotalParamVariation 2.968e-05
[0.19063545] Average Model
[0.25] Average Data
[0.05936455] Diff

Epoch 

Epoch 2398 TotalParamVariation 3.637e-05
[0.17725753] Average Model
[0.25] Average Data
[0.07274247] Diff

Epoch 2399 TotalParamVariation 1.63e-05
[0.2173913] Average Model
[0.25] Average Data
[0.0326087] Diff

Epoch 2400 TotalParamVariation 3.637e-05
[0.17725753] Average Model
[0.25] Average Data
[0.07274247] Diff

Epoch 2401 TotalParamVariation 3.972e-05
[0.17056856] Average Model
[0.25] Average Data
[0.07943144] Diff

Epoch 2402 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2403 TotalParamVariation 5.309e-05
[0.14381271] Average Model
[0.25] Average Data
[0.10618729] Diff

Epoch 2404 TotalParamVariation 2.299e-05
[0.20401338] Average Model
[0.25] Average Data
[0.04598662] Diff

Epoch 2405 TotalParamVariation 5.309e-05
[0.14381271] Average Model
[0.25] Average Data
[0.10618729] Diff

Epoch 2406 TotalParamVariation 4.306e-05
[0.1638796] Average Model
[0.25] Average Data
[0.0861204] Diff

Epoch 2407 TotalParamVariation 5.644e-05
[0.